<a href="https://colab.research.google.com/github/JoacoMunoz1/CienciaDeDatos/blob/main/Sm6_pandas_datareader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **pandas_datareader**

Esta biblioteca permirte acceder a distintos Set de Datos de forma remota.


A continuación revisaremos algunas conexiones que ofrece esta biblioteca.

In [1]:
%pip install pandas_datareader

In [2]:
from IPython.display import IFrame

# Mostrar la documentación de pandas-datareader en un iframe
IFrame('https://pandas-datareader.readthedocs.io/en/latest/py-modindex.html', width=1000, height=400)


## Información obtenida desde el Banco Mundial

In [3]:
from pandas_datareader import wb
from datetime import datetime

### Seleccionar Paises

`pandas_datareader.wb.get_countries(**kwargs)`

Información administrativa de los paises registrados en la base de datos del Banco Mundial.

In [ ]:
wb.get_countries()

In [ ]:
df_info_paises = wb.get_countries()
df_info_paises.head()

In [ ]:
# Seleccionar pais Chile
df_info_paises.query("name=='Chile'")

In [ ]:
df_info_paises[df_info_paises['name'].str.contains('hile')]

In [ ]:
df_info_paises.info()

In [ ]:
df_info_paises['region'].unique()

In [ ]:
df_paises = df_info_paises[df_info_paises['region']!= 'Aggregates']

In [ ]:
# Conocer las regiones asociadas a los paises
df_paises['region'].value_counts().reset_index()

In [ ]:
#Quitar espacios en blanco al inicio y al final
df_paises['region'] = df_paises['region'].str.strip()

In [ ]:
# Conocer las regiones asociadas a los paises
df_paises['region'].unique()

In [ ]:
df_paises.head(1)

In [ ]:
# Tabla pivot por región y nivel de ingreso
import pandas as pd
pd.pivot_table(data = df_paises, index = 'region',columns = 'incomeLevel',aggfunc='size', fill_value=0)

In [ ]:
df_paises[(df_paises['region'] == 'East Asia & Pacific') & (df_paises['incomeLevel'] == 'Low income')]

In [ ]:
df_paises[(df_paises['region'] == 'Latin America & Caribbean') & (df_paises['incomeLevel'] == 'Not classified')]

In [ ]:
df_paises[df_paises['name'].str.contains('Salvador')]

In [ ]:
# Graficar de acuerdo al nivel de Ingreso
import plotly.express as px


# Crear el gráfico coroplético básico
fig = px.choropleth(
    df_paises,
    locations='iso3c',
    color='incomeLevel',
    hover_name='name',
    projection='natural earth',
    title='Nivel de ingresos paises',
    color_continuous_scale=px.colors.sequential.Blues_r  # Cambiar la escala de colores
)

# Ajustar el tamaño de la figura
fig.update_layout(
    width=1000,  # Ancho del gráfico
    height=600,  # Altura del gráfico
    title_font_size=24  # Tamaño de la fuente del título
)

# Mostrar el gráfico
fig.show()

### Obtener las series (bases de datos) del Banco Mundial

`pandas_datareader.wb.get_indicators(**kwargs)`

Información acerca de las series del Banco Mundial.

In [ ]:
#Obtener indicadores del Banco Mundial
indicadores = wb.get_indicators()
indicadores


In [ ]:
indicadores['name'].str.contains()

In [ ]:
# Vusualizar
indicadores.query("name.str.contains(r'^[Aa]gricultural land', regex = True)")

### Descargar series (bases de datos) de distintos paises

```python
pandas_datareader.wb.download(country=None,
        indicator=None,
        start=2003,
        end=2005,
        freq=None,
        errors='warn',
        **kwargs)
```

Función que descarga información de indicadores de desarrollo


In [ ]:
start = datetime(1960, 1, 1)
end = datetime(2022, 12, 31)
id="AG.LND.AGRI.ZS"

In [ ]:
# Obtener data Tierras agrícolas (% del área de tierra)
df_ag_land = wb.download(indicator=id, start =start, end=end, country='all')
df_ag_land.head(10)

In [ ]:
# Resetear Indices
df_ag_land = df_ag_land.reset_index()
df_ag_land.head(10)

In [ ]:
df_ag_land['country'].unique()

In [ ]:
df_ag_land = df_ag_land.sort_values(['country','year'],ascending=[True,True])

In [ ]:
df_ag_land['country'].unique()

In [ ]:
regiones = list(df_paises['region'].unique())
regiones

In [ ]:
df_region = df_ag_land[df_ag_land['country'].isin(regiones)]

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import plotly.express as px

In [ ]:
px.line(data_frame=df_region,
        x='year',
        y='AG.LND.AGRI.ZS',
        color='country',
        title='Tierras agrícolas (% del área de tierra)')

In [ ]:
# Seleccionar serie PIB
start = datetime(1960, 1, 1)
end = datetime(2025, 1, 1)
indicator_id = 'NY.GDP.PCAP.KD'

In [ ]:
gdp_per_capita = wb.download(indicator=indicator_id, start =start, end=end, country='all')

In [ ]:
gdp_per_capita = gdp_per_capita.reset_index()
gdp_per_capita.head()

In [ ]:
gdp_per_capita = gdp_per_capita.sort_values(['country','year'],ascending=[True,True])

In [ ]:
gdp_per_capita_region = gdp_per_capita[gdp_per_capita['country'].isin(list(df_info_paises['region'].unique()))]

In [ ]:
gdp_per_capita_region.head(10)

In [ ]:
df_pib_region = gdp_per_capita_region[gdp_per_capita_region['country'].isin(regiones)]

In [ ]:
px.line(data_frame=df_pib_region, x='year', y='NY.GDP.PCAP.KD', color='country', title = 'PIB per-capita (US$ constante 2015)')

In [ ]:
px.line(data_frame=gdp_per_capita_region, x='year', y='NY.GDP.PCAP.KD', color='country', title = 'PIB per-capita (US$ constante 2015)')

In [ ]:

gdp_per_capita[gdp_per_capita['country'] == 'Chile']

In [ ]:
px.line(data_frame=gdp_per_capita[gdp_per_capita['country'] == 'Chile'], x='year', y='NY.GDP.PCAP.KD', color='country', title = 'PIB per-capita Chile (US$ constante 2015)')

## yfinance

In [ ]:
import pandas_datareader.data as web
import pandas as pd
import datetime as dt


In [ ]:
%pip install yfinance

In [ ]:
import yfinance as yf

In [ ]:
dat = yf.Ticker("MSFT")
dat

In [ ]:
dat.history(period='1mo')